# IMPORT & SPLIT

In [ ]:
using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, Distributions, Random, ScikitLearn, GLM, Plots

## Fonctions globales

In [ ]:
#fonction générique pour former un dataframe contenant les variables explicatives
# Array of data contient les dataframes des va explicatives
#list_of_va contient le type des données i.e ["sum" "max" ...]
# surverse contient les données de surverse
function createDataEx(array_of_data, list_of_va, dates)
    df = DataFrame(date = dates)
    
    for va in 1:length(list_of_va)
        array = array_of_data[va]
        McTavish = Array{Union{Missing, Int64}}(missing, size(df,1))
        Bellevue = Array{Union{Missing, Int64}}(missing, size(df,1))
        Assomption = Array{Union{Missing, Int64}}(missing, size(df,1))
        Trudeau = Array{Union{Missing, Int64}}(missing, size(df,1))
        StHubert = Array{Union{Missing, Int64}}(missing, size(df,1))
        
        for i=1:size(df,1)
            ind = findfirst(array[:,:date] .== df[i,:date])
            McTavish[i] = array[ind,:McTavish]
            Bellevue[i] = array[ind,:Bellevue]
            Assomption[i] = array[ind,:Assomption]
            Trudeau[i] = array[ind,:Trudeau]
            StHubert[i] = array[ind,:StHubert]
        end
        
        df[!,Symbol(list_of_va[va] * "McTavish")] = McTavish
        df[!,Symbol(list_of_va[va] * "Bellevue")] = Bellevue   
        df[!,Symbol(list_of_va[va] * "Assomption")] = Assomption   
        df[!,Symbol(list_of_va[va] * "Trudeau")] = Trudeau   
        df[!,Symbol(list_of_va[va] * "StHubert")] = StHubert
    end
    
    return df
end

#fonction pour recuperer seulement les donnees d'un ouvrage
#fonction pour filtrer un dataframe selon l<ouvrage
function getOuvrage(data, ouvrage)
    return filter(row -> row.NO_OUVRAGE == ouvrage, data)
end

#fonction pour recuperer seulement les donnees contenu a bonnes dates
function getDataFromDates(df, dates)
    return filter(row -> row.date in dates, df)
end

"""
    splitdataframe(df::DataFrame, p::Real)

Partitionne en un ensemble d'entraînement et un ensemble de validation un DataFrame.

### Arguments
- `df::DataFrame` : Un DataFrame
- `p::Real` : La proportion (entre 0 et 1) de données dans l'ensemble d'entraînement.

### Détails

La fonction renvoie deux DataFrames, un pour l'ensemble d'entraînement et l'autre pour l'ensemble de validation.

### Exemple

\```
 julia> splitdataframe(df, p.7)
\```

"""
function splitdataframe(df::DataFrame, p::Real)
   @assert 0 <= p <= 1 
    
    n = size(df,1)
    
    ind = shuffle(1:n)
    
    threshold = Int64(round(n*p))
    
    indTrain = sort(ind[1:threshold])
    
    indTest = setdiff(1:n,indTrain)
    
    dfTrain = df[indTrain,:]
    dfTest = df[indTest,:]
    
    return dfTrain, dfTest
    
end

### Fonction pour la conversion d'un dataframe en matrix/array
On ignore les dates et les ouvrages si necessaire

In [ ]:
function toArray(df::DataFrame)
    
    if :NO_OUVRAGE in names(df)
        return convert(Matrix, df[:, Not([:date, :NO_OUVRAGE])])
    elseif :date in names(df)
        return convert(Matrix, df[:, Not(:date)])
    else
        return convert(Matrix, df)
    end
end
function toArray(df::DataFrameRow)
    
    if :NO_OUVRAGE in names(df)
        return convert(Vector, df[Not([:date, :NO_OUVRAGE])])
    elseif :date in names(df)
        return convert(Vector, df[Not(:date)])
    else
        return convert(Vector, df)
    end
end 

## Lecture des fichiers des variables explicatives et des surverses

In [ ]:
x_train_max = CSV.read("./data/new_datasets/precipitation_daily_max/x_train.csv");
x_pred_max = CSV.read("./data/new_datasets/precipitation_daily_max/x_pred.csv");

In [ ]:
x_train_sum = CSV.read("./data/new_datasets/precipitation_daily_sum/x_train.csv");
x_pred_sum = CSV.read("./data/new_datasets/precipitation_daily_sum/x_pred.csv");

In [ ]:
x_sum_last_2 = CSV.read("./data/new_datasets/sum_day_last_2.csv")
x_train_sum_last_2 = filter(row -> Year(row[:date]) != Year(2019), x_sum_last_2)
x__pred_sum_last_2  = filter(row -> Year(row[:date]) == Year(2019), x_sum_last_2);

In [ ]:
x_max_2 = CSV.read("./data/new_datasets/max_precipitation_day_split/maxPrecBy2hours.csv")
x_train_max_2 = filter(row -> Year(row[:date]) != Year(2019), x_max_2)
x_pred_max_2  = filter(row -> Year(row[:date]) == Year(2019), x_max_2);

In [ ]:
x_max_3 = CSV.read("./data/new_datasets/max_precipitation_day_split/maxPrecBy3hours.csv")
x_train_max_3 = filter(row -> Year(row[:date]) != Year(2019), x_max_3)
x_pred_max_3  = filter(row -> Year(row[:date]) == Year(2019), x_max_3);

In [ ]:
x_max_4 = CSV.read("./data/new_datasets/max_precipitation_day_split/maxPrecBy4hours.csv")
x_train_max_4 = filter(row -> Year(row[:date]) != Year(2019), x_max_4)
x_pred_max_4  = filter(row -> Year(row[:date]) == Year(2019), x_max_4);

In [ ]:
x_max_6 = CSV.read("./data/new_datasets/max_precipitation_day_split/maxPrecBy6hours.csv")
x_train_max_6 = filter(row -> Year(row[:date]) != Year(2019), x_max_6)
x_pred_max_6  = filter(row -> Year(row[:date]) == Year(2019), x_max_6);

In [ ]:
x_max_8 = CSV.read("./data/new_datasets/max_precipitation_day_split/maxPrecBy8hours.csv")
x_train_max_8 = filter(row -> Year(row[:date]) != Year(2019), x_max_8)
x_pred_max_8  = filter(row -> Year(row[:date]) == Year(2019), x_max_8);

In [ ]:
x_max_12 = CSV.read("./data/new_datasets/max_precipitation_day_split/maxPrecBy12hours.csv")
x_train_max_12 = filter(row -> Year(row[:date]) != Year(2019), x_max_12)
x_pred_max_12  = filter(row -> Year(row[:date]) == Year(2019), x_max_12);

### Deciding which variables to use

In [ ]:
y_train = CSV.read("./data/new_datasets/surverse_list.csv");
x_train = createDataEx([x_train_max, x_train_sum, x_train_max_2, x_train_max_3, x_train_max_4, x_train_max_2], ["max", "sum", "max2", "max3", "max4"], x_train_max_2[:, :date])
x_pred = createDataEx([x_pred_max, x_pred_sum, x_pred_max_2, x_pred_max_3, x_pred_max_4, x_pred_max_2], ["max", "sum", "max2", "max3", "max4"], x_pred_max_2[:, :date])
dropmissing!(x_pred)

In [ ]:
# Ouvrages qui nous interesse
ouvrages = ["3260-01D", "3350-07D", "4240-01D", "4350-01D", "4380-01D"]

# PLOT

In [ ]:
histogram(convert(Matrix, x_train[:, Not(:date)]))

We can observe that most data is skewed towards 0cm with some extreme cases over 2000cm (outliers)

we need to preprocess the data with a function to reduce this gap

$ f(x) = ln(x+1) $

In [ ]:
x_values = DataFrame([log.(col.+1) for col = eachcol(x_train[:, Not(:date)])])
display(histogram(convert(Matrix, x_values)))
display(describe(x_values, :mean, :std, :min, :q25, :median, :q75, :max))
x_train_log = log.(x_train[:, Not(:date)] .+ 1)
x_pred_log = log.(x_pred[:, Not(:date)] .+ 1);

# MULTICOLINEARITY

In [ ]:
CUTOFF = 7

@sk_import decomposition : PCA
pca = PCA()
x_train_matrix = pca.fit_transform(toArray(x_train))

explained_variance = cumsum(pca.explained_variance_ratio_)

Plots.plot(
    1:length(pca.explained_variance_ratio_),
    explained_variance,
    ylims=(0, 1.0),
    xlabel="Number of components",
    ylabel="Cumulative sum of explained Variance"
)
display(
    annotate!([(12.5, .4, 
        text("Cutoff at $(CUTOFF) variables which is $(round(explained_variance[CUTOFF], digits=5))% of the total variance",10)
    )])
)

x_train_pca = DataFrame(
    date=x_train[!,:date],
    x₁=x_train_matrix[:,1],
    x₂=x_train_matrix[:,2],
    x₃=x_train_matrix[:,3],
    x₄=x_train_matrix[:,4],
    x₅=x_train_matrix[:,5],
    x₆=x_train_matrix[:,6],
    x₇=x_train_matrix[:,7],
);

# Data Split

Aiming for

- 80% TRAIN
- 20% TEST

**TODO: enable use of different datasets (PCA with and without log)**

In [ ]:
Y = y_train
X = x_pred
X_PCA = x_train_pca

function dataSplit(ouvrage, X_train = X_PCA) #HERE
    y = getOuvrage(Y, ouvrage)
    y_train, y_test = splitdataframe(y, .80)

    x_train = getDataFromDates(X_train, y_train.date)
    y_train = getDataFromDates(y_train, x_train.date)

    x_test = getDataFromDates(X_train, y_test.date)
    y_test = getDataFromDates(y_test, x_test.date)
    
    return x_train, x_test, y_train, y_test
end

# MODELS

In [ ]:
@sk_import metrics: f1_score;

In [ ]:
function train_on_all_ouvrages(model_contructor::Function)
    pred = []
    test = []
    for ouvrage in ouvrages
        x_train, x_test, y_train, y_test = dataSplit(ouvrage)
        
        model = model_contructor(x_train, y_train)
        
        prediction = model.predict(toArray(x_test));
        
        push!(test, toArray(y_test))
        push!(pred, prediction)
    end
    
    pred = collect(Iterators.flatten(pred))
    test = collect(Iterators.flatten(test))
    return f1_score(pred,test)
end

## Logistic Regression

In [ ]:
@sk_import linear_model: LogisticRegression
function modeleLogistique(x_train, y_train) 
    model = LogisticRegression(
        penalty="l2",
        solver="liblinear",
        random_state=234,
    )
    model.fit(toArray(x_train), toArray(y_train));
    return model
end

## Modele de l'arbre de Decision

In [ ]:
@sk_import tree: DecisionTreeClassifier
function modeleArbreDecisif(x_train, y_train)
    model = DecisionTreeClassifier(
        random_state=234,
    )
    model.fit(toArray(x_train), toArray(y_train));
    return model
end

# Train All Models

In [ ]:
Iterations=100
display("$(Iterations)  iterations")

models = [
    modeleLogistique,
    modeleArbreDecisif,
];

# TODO -> VALIDATE DATASETS
for m in models
    
    score=[]
    for i = 1:Iterations
        f1 = train_on_all_ouvrages(m);
        push!(score,f1)
    end
    f1 = mean(score)
    
    f1 = round(f1, digits=4)
    m = string(m)

    display("f1 = $(f1) with $(m)")
end

Results show:

Logistic = .69

Trees = .61

## Mise en place des valeurs cherchees

In [ ]:
test = CSV.read("data/test.csv")
df = test[:, [:NO_OUVRAGE, :DATE]]
rename!(df, :DATE=>:date)
X = join(df, x_pred, on=:date)

In [ ]:
function predictLogit(df, modeles)
    dataframe = DataFrame(NO_OUVRAGE=df.NO_OUVRAGE, x₁=df.sumMcTavish,x₂=df.sumBellevue,x₃=df.sumAssomption,x₄=df.sumTrudeau,x₅=df.sumStHubert,x₆=df.maxMcTavish,x₇=df.maxBellevue,x₈=df.maxAssomption,x₉=df.maxTrudeau,x₁₀=df.maxStHubert)
    predictions = []
    for x in eachrow(dataframe)
        modele = filter(row -> row.NO_OUVRAGE == x.NO_OUVRAGE, modeles).modele[1]
        predictions = vcat(predictions, GLM.predict(modele, DataFrame(x[Not([:NO_OUVRAGE])])))
    end
    return round.(predictions)
end

In [ ]:
function predictTree(dataframe, modeles)
    predictions = []
    for x in eachrow(dataframe)
        modele = filter(row -> row.NO_OUVRAGE == x.NO_OUVRAGE, modeles).modele[1]
        predictions = vcat(predictions, modele.predict([toArray(x)]))
    end
    return round.(predictions)
end

In [ ]:
predictionsTree = predictTree(X, modelesTree)

In [ ]:
predictionsLogit = predictLogit(X, modelesLogit)

In [ ]:
#Nombre de diff entre Tree et Logit
sum(abs.(predictionsTree-predictionsLogit))

# Création du fichier de prédictions pour soumettre sur Kaggle

Dans ce cas-ci, nous prédirons une surverse avec une prediction logistique

In [ ]:
# Création du fichier sampleSubmission.csv pour soumettre sur Kaggle
ID = test[:,:NO_OUVRAGE].*"_".*string.(test[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=predictionsTree)
CSV.write("sampleSubmission.csv",sampleSubmission)

# Vous pouvez par la suite déposer le fichier sampleSubmission.csv sur Kaggle.